Indel candidacy is determined by observing sufficient counts of reads supporting the indel vs. all reads supporting the indel or an alternate allele at that locus. Sufficient counts in this case is determined by rejecting a null hypothesis of a per read indel error process occuring at the indel error rate (p) for a given alpha, as originally setup by Mitch Bekritsky:

In [10]:
GetCountsToRejectNull <- function(alpha, n_trials, p) {
  counts <- 1 + qbinom(alpha, n_trials, p, lower.tail = FALSE)
  return (counts)
}

To compare models we setup some counts table printing boilerplate:

In [11]:
coverageLevels <- 1:10*10
DumpCov <- function() {
  cat("coverage")
  for (cov in coverageLevels){
    cat(paste("\t",cov))
  }
  cat("\n")
}

PrintCountsToReject <- function(alpha, errorRate, label) {
  cat(label)
  for (cov in coverageLevels){
    cat(paste("\t",GetCountsToRejectNull(alpha, cov, errorRate)))
  }
  cat("\n")
}

The v2.7.x candidacy model is:

In [12]:
alpha <- 1e-9
DumpCov()
PrintCountsToReject(alpha, 5e-5, "nonSTRIndel")
PrintCountsToReject(alpha, 3e-4, "hpol16+Indel")

coverage	 10	 20	 30	 40	 50	 60	 70	 80	 90	 100
nonSTRIndel	 3	 3	 3	 4	 4	 4	 4	 4	 4	 4
hpol16+Indel	 4	 4	 4	 4	 5	 5	 5	 5	 5	 5


You can see how this model merges together the previous 10% cutoff behavior chosen for old versions of the germline model, with a plateauing evidence requirement at high coverage, as we would like for somatic models.

Attempting to make the candidate model more "consistent" by using the same indel error rates we use for the germline indel likelihood generation tends to reduce F-scores compared to using the old candidacy model. Looking at the tables suggests why.

Here is the first attemp to just use the indel error rates from v2.7.x (so the same rates as shown above, times 100). Alpha has been scaled to produce a similar number of candidates on a ~35x example:

In [13]:
alpha <- 1e-2
DumpCov()
PrintCountsToReject(alpha, 5e-3, "nonSTRIndel")
PrintCountsToReject(alpha, 3e-2, "hpol16+Indel")
cat("\n")

coverage	 10	 20	 30	 40	 50	 60	 70	 80	 90	 100
nonSTRIndel	 2	 2	 2	 3	 3	 3	 3	 3	 4	 4
hpol16+Indel	 3	 4	 5	 5	 6	 6	 7	 8	 8	 9



Very different response as a function of depth, as expected. This doesn't suggest to me that our indel error rates are wrong, so much as the null rejection test isn't what we really want. In v2.7.x the combination of the "not-quite-right" rates and the "not-quite-right" test happens to have all the properties we're looking for.

UPdating indel error rates to the ones we actually estimate from data (shown here is the geometric avg of Nano and PCR-free), doesn't really change the problems with this approach, as is also observed in similar F-score drops:

In [14]:
alpha <- 1e-2
DumpCov()
PrintCountsToReject(alpha, 1.2e-2, "nonSTRIndel")
PrintCountsToReject(alpha, 4.9e-2, "hpol16+Indel")

coverage	 10	 20	 30	 40	 50	 60	 70	 80	 90	 100
nonSTRIndel	 2	 3	 3	 4	 4	 4	 5	 5	 5	 5
hpol16+Indel	 4	 5	 6	 7	 8	 8	 9	 10	 11	 11
